# Importing data from a sample csv


In [61]:
# not sure why the instructor suggests using this csv method, when pandas can import an xls file directly
import csv

# This is just to show that I am able to read a csv file
with open("sample-data.csv", "r") as file:
    reader = csv.reader(file, delimiter=",")
    for row in reader:
        print(row)

['first_name', 'last_name', 'email', 'phone']
['John', 'Doe', 'john@gmail.com', '1234567890']
['Kelly', 'Smith', 'ks@gmail.com', '3444583989']
['Roger', 'Moore', 'moore@gmail.com', '8876567890']
['Edward', 'Smith', 'esmith@gmail.com', '2222223332']
['Lenny', 'Warren', 'coollenny@yahoo.com', '3333333333']
['Trish', 'Lopez', 'lolo@gmail.com', '3432223833']


# Create DDL to create the database and tables


In [62]:
# first install the mysql-connector-python package using the following command
#   pip install mysql-connector-python

import mysql.connector
import yaml

db = yaml.safe_load(open("secrets.yaml"))

# Establish a connection to the MySQL server
cnx = mysql.connector.connect(
    host=db["DB_HOST"],
    user=db["DB_USER"],  # both root and andrewcbuensalida works
    password=db["DB_PASSWORD"],
)

# Create a cursor object
cursor = cnx.cursor()

# Drop the database if it exists
cursor.execute("DROP DATABASE IF EXISTS mrts")

# Create a database
cursor.execute("CREATE DATABASE mrts")

# Use the created database
cursor.execute("USE mrts")

# Create the NAICS code table
cursor.execute(
    """
    CREATE TABLE naics_code (
        id INT AUTO_INCREMENT PRIMARY KEY,
        naics_code VARCHAR(255)
    )
"""
)

# Create the kind of business table
cursor.execute(
    """
    CREATE TABLE kind_of_business (
        id INT AUTO_INCREMENT PRIMARY KEY,
        kind_of_business VARCHAR(255)
    )
"""
)

# Create the sales table
cursor.execute(
    """
    CREATE TABLE sales (
        id INT AUTO_INCREMENT PRIMARY KEY,
        sales_amount DECIMAL(10, 2),
        month INT,
        year INT,
        kind_of_business_id INT,
        FOREIGN KEY (kind_of_business_id) REFERENCES kind_of_business(id)
    )
"""
)

# Create the NAICS code-kind of business association table
cursor.execute(
    """
    CREATE TABLE naics_kind_of_business_association (
        naics_code_id INT,
        kind_of_business_id INT,
        FOREIGN KEY (naics_code_id) REFERENCES naics_code(id),
        FOREIGN KEY (kind_of_business_id) REFERENCES kind_of_business(id)
    )
"""
)

# Importing data from mrts csv year 2020 then save into mysql


In [63]:
# Insert kind of business into sql table
for i in range(1992, 2021):
    with open(f"mrtssales92-present/{i}.csv", "r") as file:
        reader = csv.reader(file, delimiter=",")
        for index, row in enumerate(reader):
            if index >= 6 and index <= 70:  # not adjusted industry categories
                # Check if the kind of business already exists
                query = (
                    "SELECT COUNT(*) FROM kind_of_business WHERE kind_of_business = %s"
                )
                data = (row[1],)

                # Execute the query
                cursor.execute(query, data)

                # Fetch the result
                result = cursor.fetchone()

                # Extract the count from the result
                count = result[0]

                # If the kind of business doesn't exist, insert it into the table
                if count == 0:
                    # Insert kind of business in table
                    query = (
                        "INSERT INTO kind_of_business (kind_of_business) VALUES (%s)"
                    )
                    data = (row[1],)

                    # Execute the query
                    cursor.execute(query, data)

                    # Commit the transaction
                    cnx.commit()

In [64]:
# Insert NAICS codes into table
for i in range(1992, 2021):
    with open(f"mrtssales92-present/{i}.csv", "r") as file:
        reader = csv.reader(file, delimiter=",")
        for index, row in enumerate(reader):
            if index >= 6 and index <= 70 and row[0]:  # not adjusted industry categories
                codes = row[0].split(",")

                for code in codes:
                    # Check if the code already exists in the table
                    query = "SELECT COUNT(*) FROM naics_code WHERE naics_code = %s"
                    data = (code,)

                    # Execute the query
                    cursor.execute(query, data)

                    # Fetch the result
                    result = cursor.fetchone()

                    # Extract the count from the result
                    count = result[0]

                    # If the code doesn't exist, insert it into the table
                    if count == 0:
                        query = "INSERT INTO naics_code (naics_code) VALUES (%s)"
                        data = (code,)

                        # Execute the query
                        cursor.execute(query, data)

                        # Commit the transaction
                        cnx.commit()

In [65]:
# Insert NAICS code-kind of business associations into table
for i in range(1992, 2021):
    with open(f"mrtssales92-present/{i}.csv", "r") as file:
        reader = csv.reader(file, delimiter=",")
        for index, row in enumerate(reader):
            if index >= 6 and index <= 70 and row[0]:  # not adjusted industry categories
                codes = row[0].split(",")

                for code in codes:
                    # Get the NAICS code ID
                    query = "SELECT id FROM naics_code WHERE naics_code = %s"
                    data = (code,)

                    # Execute the query
                    cursor.execute(query, data)

                    # Fetch the result
                    result = cursor.fetchone()

                    # Extract the ID from the result
                    naics_code_id = result[0]

                    # Get the kind of business ID
                    query = "SELECT id FROM kind_of_business WHERE kind_of_business = %s"
                    data = (row[1],)

                    # Execute the query
                    cursor.execute(query, data)

                    # Fetch the result
                    result = cursor.fetchone()

                    # Extract the ID from the result
                    kind_of_business_id = result[0]

                    # Insert the association into the table
                    query = "INSERT INTO naics_kind_of_business_association (naics_code_id, kind_of_business_id) VALUES (%s, %s)"
                    data = (naics_code_id, kind_of_business_id)

                    # Execute the query
                    cursor.execute(query, data)

                    # Commit the transaction
                    cnx.commit()

In [66]:
# Insert sales data into table. This takes about 1.5 minutes to run.
for i in range(1992, 2021):
    with open(f"mrtssales92-present/{i}.csv", "r") as file:
        reader = csv.reader(file, delimiter=",")
        for index, row in enumerate(reader):
            if index == 0:
                title = row[0]

            if index >= 6 and index <= 70:  # not adjusted industry categories
                #  Get the kind of business id from sql
                query = "SELECT id FROM kind_of_business WHERE kind_of_business = %s"
                data = (row[1],)

                # Execute the query
                cursor.execute(query, data)

                # Fetch the result
                result = cursor.fetchone()

                # Extract the id from the result
                if result:
                    kind_of_business_id = result[0]
                else:
                    print(
                        "No matching id found"
                    )  # maybe could add a new kind of business here

                # sales from january to december
                sales = row[2:-1]

                for index, sales_amount in enumerate(sales):
                    month = index + 1
                    year = int(title.split(":")[-1])
                    if (
                        sales_amount != "(S)" and sales_amount != "(NA)" and sales_amount
                    ):  # There are some data that have a string (S) instead of a proper sales amount, and some are empty. Just don't insert these sales.

                        sales_amount_float = float(sales_amount.replace(",", ""))

                        data = [kind_of_business_id, sales_amount_float, month, year]
                        print(data)

                        # The SQL query to insert the data
                        query = "INSERT INTO sales (kind_of_business_id, sales_amount, month, year) VALUES (%s, %s, %s, %s)"

                        # Execute the query
                        cursor.execute(query, data)

                        # Commit the transaction
                        cnx.commit()

[1, 146376.0, 1, 1992]
[1, 147079.0, 2, 1992]
[1, 159336.0, 3, 1992]
[1, 163669.0, 4, 1992]
[1, 170068.0, 5, 1992]
[1, 168663.0, 6, 1992]
[1, 169890.0, 7, 1992]
[1, 170364.0, 8, 1992]
[1, 164617.0, 9, 1992]
[1, 173655.0, 10, 1992]
[1, 171547.0, 11, 1992]
[1, 208838.0, 12, 1992]
[2, 116565.0, 1, 1992]
[2, 115862.0, 2, 1992]
[2, 124200.0, 3, 1992]
[2, 127587.0, 4, 1992]
[2, 133608.0, 5, 1992]
[2, 130274.0, 6, 1992]
[2, 132076.0, 7, 1992]
[2, 134928.0, 8, 1992]
[2, 128734.0, 9, 1992]
[2, 136917.0, 10, 1992]
[2, 139187.0, 11, 1992]
[2, 175771.0, 12, 1992]
[3, 134277.0, 1, 1992]
[3, 135499.0, 2, 1992]
[3, 147064.0, 3, 1992]
[3, 151226.0, 4, 1992]
[3, 156619.0, 5, 1992]
[3, 155236.0, 6, 1992]
[3, 156016.0, 7, 1992]
[3, 156539.0, 8, 1992]
[3, 151477.0, 9, 1992]
[3, 159992.0, 10, 1992]
[3, 158483.0, 11, 1992]
[3, 195350.0, 12, 1992]
[4, 104466.0, 1, 1992]
[4, 104282.0, 2, 1992]
[4, 111928.0, 3, 1992]
[4, 115144.0, 4, 1992]
[4, 120159.0, 5, 1992]
[4, 116847.0, 6, 1992]
[4, 118202.0, 7, 1992]
[4

# Explore trends


# Explore percentage change


# Explore rolling time windows


In [67]:
# # Close the cursor and connection
cursor.close()
cnx.close()